# Capstone Notebook

This Notebook is created for the Applied Data Science Capstone project

In [3]:
import pandas as pd
import numpy as np


In [4]:
print('Hello Capstone Project Course')

Hello Capstone Project Course


Importing packages needed

In [5]:
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json


from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


In [6]:
data_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

## Scaper

In [15]:
from bs4 import BeautifulSoup as bs
ht_temp = requests.get(data_url).text
ht = bs(ht_temp,'lxml')

In [54]:
ht_table = ht.find('table')
ht_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>Ea

In [73]:
import csv
csv_file = open('scrape.csv','w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['PostalCode','Borough','Neighborhood'])

for tr in ht_table.find_all('tr'):
    for td in tr.find_all('td'):
        try:
            PostalCode = td.text
            print(PostalCode)
        except Exception as e:
            PostalCode = None   
        try:
            Borough = td.text
            print(Borough)
        except Exception as e:
             Borough = None            
        try:        
            Neighborhood = td.text
            print(Neighborhood)
        except Exception as e:
            Neighborhood = None
        csv_writer.writerow([PostalCode,Borough,Neighborhood])
csv_file.close()

M1A

M1A

M1A

Not assigned

Not assigned

Not assigned







M2A

M2A

M2A

Not assigned

Not assigned

Not assigned







M3A

M3A

M3A

North York

North York

North York

Parkwoods

Parkwoods

Parkwoods

M4A

M4A

M4A

North York

North York

North York

Victoria Village

Victoria Village

Victoria Village

M5A

M5A

M5A

Downtown Toronto

Downtown Toronto

Downtown Toronto

Regent Park / Harbourfront

Regent Park / Harbourfront

Regent Park / Harbourfront

M6A

M6A

M6A

North York

North York

North York

Lawrence Manor / Lawrence Heights

Lawrence Manor / Lawrence Heights

Lawrence Manor / Lawrence Heights

M7A

M7A

M7A

Downtown Toronto

Downtown Toronto

Downtown Toronto

Queen's Park / Ontario Provincial Government

Queen's Park / Ontario Provincial Government

Queen's Park / Ontario Provincial Government

M8A

M8A

M8A

Not assigned

Not assigned

Not assigned







M9A

M9A

M9A

Etobicoke

Etobicoke

Etobicoke

Islington Avenue

Islington Avenue

Islington Avenue

M1B


M2X

M2X

M2X

Not assigned

Not assigned

Not assigned







M3X

M3X

M3X

Not assigned

Not assigned

Not assigned







M4X

M4X

M4X

Downtown Toronto

Downtown Toronto

Downtown Toronto

St. James Town / Cabbagetown

St. James Town / Cabbagetown

St. James Town / Cabbagetown

M5X

M5X

M5X

Downtown Toronto

Downtown Toronto

Downtown Toronto

First Canadian Place / Underground city

First Canadian Place / Underground city

First Canadian Place / Underground city

M6X

M6X

M6X

Not assigned

Not assigned

Not assigned







M7X

M7X

M7X

Not assigned

Not assigned

Not assigned







M8X

M8X

M8X

Etobicoke

Etobicoke

Etobicoke

The Kingsway / Montgomery Road  / Old Mill North

The Kingsway / Montgomery Road  / Old Mill North

The Kingsway / Montgomery Road  / Old Mill North

M9X

M9X

M9X

Not assigned

Not assigned

Not assigned







M1Y

M1Y

M1Y

Not assigned

Not assigned

Not assigned







M2Y

M2Y

M2Y

Not assigned

Not assigned

Not assigned







M3Y

M3Y



In [74]:
Neighborhoods = pd.read_csv('scrape.csv')
Neighborhoods

,PostalCode,Borough,Neighborhood
0,M1A\r\n,M1A\r\n,M1A\r\n
1,Not assigned\r\n,Not assigned\r\n,Not assigned\r\n
2,\r\n,\r\n,\r\n
3,M2A\r\n,M2A\r\n,M2A\r\n
4,Not assigned\r\n,Not assigned\r\n,Not assigned\r\n
5,\r\n,\r\n,\r\n
6,M3A\r\n,M3A\r\n,M3A\r\n
7,North York\r\n,North York\r\n,North York\r\n
8,Parkwoods\r\n,Parkwoods\r\n,Parkwoods\r\n
9,M4A\r\n,M4A\r\n,M4A\r\n
